# Depth estimator

> Depth estimator


- skip_showdoc: true
- skip_exec: true

In [ ]:
import torch
import torchvision
from torchvision import transforms, datasets, utils
import urllib.request
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
############## TENSORBOARD ########################
import sys
from torch.utils.tensorboard import SummaryWriter
# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter()
###################################################

In [ ]:
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
urllib.request.urlretrieve(url, filename)

In [ ]:
#model_type = "DPT_Large"     # MiDaS v3 - Large     (highest accuracy, slowest inference speed)
#model_type = "DPT_Hybrid"   # MiDaS v3 - Hybrid    (medium accuracy, medium inference speed)
model_type = "MiDaS_small"  # MiDaS v2.1 - Small   (lowest accuracy, highest inference speed)

midas = torch.hub.load("intel-isl/MiDaS", model_type)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()


In [ ]:
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")


In [ ]:
if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

transform

In [ ]:
img = cv2.imread(filename)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img.shape

In [ ]:
input_batch = transform(img).to(device)
input_batch.shape

In [ ]:
input_batch.shape[-2]

In [ ]:
input_batch.squeeze?

In [ ]:
input_batch.squeeze(1).shape

In [ ]:
input_batch1 = input_batch.to('cpu')
input_batch1 = input_batch1.squeeze()
input_batch1 = input_batch1.numpy().transpose(1, 2, 0)
input_batch1.shape

In [ ]:
input_batch2 = transform(input_batch1).to(device)

In [ ]:
with torch.no_grad():
    prediction = midas(input_batch)

    prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1),
        size=img.shape[:2],
        mode="bicubic",
        align_corners=False,
    ).squeeze()



In [ ]:
import torch

# Assuming img_tensor is your image tensor
img_range = (torch.max(prediction) - torch.min(prediction))

# Rescale the tensor to the range [0, 1]
prediction = ((prediction - torch.min(prediction)) / img_range) * 255

# Convert the tensor to integers (rounding) and change data type to uint8
prediction = prediction.round().to(torch.uint8)

output = prediction.cpu().numpy()

In [ ]:
output.shape

In [ ]:
plt.imshow(img)
plt.axis("off")

In [ ]:
plt.imshow(output, cmap='gray')
plt.axis("off")
# plt.show()

In [ ]:
plt.imshow(output, cmap='gray')
plt.axis("off")
# plt.show()

In [ ]:
############## TENSORBOARD ########################
writer.add_image('mnist_images', img,
                 global_step=0,
                 dataformats='HWC')
writer.add_image('output', output,
                 global_step=0,
                 dataformats='HW')
writer.flush()
#sys.exit()
###################################################

In [ ]:
writer.close()